In [3]:
import pandas as pd
import os
import jdatetime
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

In [7]:
# Specify the path of the parent directory
parent_directory = 'tax'

# List all files and directories in the parent directory
all_items = os.listdir(parent_directory)

# Filter out only directories
directories = [item for item in all_items if os.path.isdir(os.path.join(parent_directory, item))]


for folder in directories:
    # Specify the directory
    directory = f"tax/{folder}/"

    # List all files and directories in the specified directory
    files_and_dirs = os.listdir(directory)

    # If you only want files, not directories, you can filter the list
    files = [f for f in files_and_dirs if os.path.isfile(os.path.join(directory, f))]
    for file in files:
        print(file)

        file_path = f'tax/{folder}/{file}'

        df = pd.read_csv(file_path)
        df['Date'] = pd.to_datetime(df['Date']).dt.date
        # Function to convert Gregorian date to Jalali date
        def gregorian_to_jalali(gregorian_date):
            return jdatetime.date.fromgregorian(date=gregorian_date)

        # Apply the conversion function to your datetime column
        df['jalali_date'] = df['Date'].apply(gregorian_to_jalali)


        if file == 'tether.csv':
            df['crypto_daily_price'] = 1
        elif file == 'eth.csv':
            df_crypto = pd.read_csv('tax/Ethereum Historical Data.csv')
            df_crypto['Date'] = pd.to_datetime(df_crypto['Date']).dt.date
            df = pd.merge(df, df_crypto[['Date', 'crypto_daily_price']], on='Date', how='left')
            df['Amount'] = df['Amount'] / 1000000000
        elif file == 'btc.csv':
            df_crypto = pd.read_csv('tax/Bitcoin Historical Data.csv')
            df_crypto['Date'] = pd.to_datetime(df_crypto['Date']).dt.date
            df = pd.merge(df, df_crypto[['Date', 'crypto_daily_price']], on='Date', how='left')
            df['Amount'] = df['Amount'] / 100000000
            df_crypto.to_excel('df_crypto.xlsx', index=False)
            df.to_excel('df.xlsx', index=False)

        df_price = pd.read_excel('tax/DOLLAR-14020824-.xlsx')
        df_price['Date'] = pd.to_datetime(df_price['Date']).dt.date

        df = pd.merge(df, df_price[['Date', 'dollor_price']], on='Date', how='left')
        
        fee = 0.005
        tax_ratio = 0.25
        Circulation_coefficient = 1
        
        df['trx_IRR'] = df['dollor_price'] * df['crypto_daily_price'] * df['Amount']
        df['trx_dollor'] = df['crypto_daily_price'] * df['Amount']
        df['calculated_tax'] = df['dollor_price']* df['Amount'] * df['crypto_daily_price'] * fee * tax_ratio * Circulation_coefficient

        # Calculate the sum of the 'calculated_tax' column
        total_sum_calculated_tax = df['calculated_tax'].sum()
        total_sum_trx = df['Amount'].sum()
        total_sum_trxـd = df['trx_dollor'].sum()
        total_sum_trx_irr = df['trx_IRR'].sum()

        # Calculate the total count of the 'calculated_tax' column
        total_count = df['calculated_tax'].count()


        # Create the full file path
        output_file_name = file.split('.')[0] + '.xlsx'
        full_file_path = f'taxes_output/{folder}--{output_file_name}'

        # Write df to an Excel file
        df.to_excel(full_file_path, index=False, sheet_name='data')

        # Create a new DataFrame for the overview
        overview_df = pd.DataFrame({'مجموع مالیات': [total_sum_calculated_tax], 'مجموع حجم کریپتویی تراکنش‌های آدرس': [total_sum_trx], 'مجموع ارزش ریال تراکنش‌های آدرس': [total_sum_trx_irr], 'مجموع دلاری تراکنش‌های آدرس': [total_sum_trxـd],'تعداد تراکنش‌ها': [total_count]})

        # Create an ExcelWriter object
        with pd.ExcelWriter(full_file_path, mode='a', engine='openpyxl') as writer:
            # Write the overview DataFrame to a new sheet named 'overview'
            overview_df.to_excel(writer, sheet_name='overview', index=False)

        # Create a new DataFrame for the overview
        pram_df = pd.DataFrame({'کارمزد صرافی': [fee], 'نرخ گردش دارایی': [Circulation_coefficient], 'ضریب مالیات بر درآمد': [tax_ratio]})

        # Create an ExcelWriter object
        with pd.ExcelWriter(full_file_path, mode='a', engine='openpyxl') as writer:
            # Write the overview DataFrame to a new sheet named 'overview'
            pram_df.to_excel(writer, sheet_name='parameters', index=False)
        
        # Function to extract the Jalali year from a jdatetime.date object
        def extract_jalali_year(jalali_date):
            return jalali_date.year

        # Assuming df['Date'] contains Jalali dates as jdatetime.date objects
        df['JalaliYear'] = df['jalali_date'].apply(extract_jalali_year)

        # Group by Jalali year and calculate sum
        grouped = df.groupby('JalaliYear')['calculated_tax'].sum()

        # Create a new DataFrame from the grouped data
        sums_by_jalali_year_df = grouped.reset_index()
        sums_by_jalali_year_df.columns = ['JalaliYear', 'TotalSum']

        with pd.ExcelWriter(full_file_path, mode='a', engine='openpyxl') as writer:
            # Write the overview DataFrame to a new sheet named 'overview'
            sums_by_jalali_year_df.to_excel(writer, sheet_name='yearly', index=False)
        
        # Load the workbook and select the worksheet
        workbook = load_workbook(full_file_path)

        # تابعی برای اعمال تغییرات روی هر شیت
        def format_sheet(sheet):
            # تنظیم رنگ ردیف‌ها
            for row in range(2, sheet.max_row + 1):
                for cell in sheet[row]:
                    if row % 2 == 0:
                        cell.fill = PatternFill(start_color='D3D3D3', end_color='D3D3D3', fill_type='solid')

            # تنظیم عرض ستون‌ها
            for column_cells in sheet.columns:
                length = max(len(str(cell.value)) for cell in column_cells)
                sheet.column_dimensions[column_cells[0].column_letter].width = length

        # اعمال تغییرات روی هر شیت
        for sheet_name in workbook.sheetnames:
            format_sheet(workbook[sheet_name])


        # Save the workbook
        workbook.save(full_file_path)


tether.csv
eth.csv
tether.csv
tether.csv
eth.csv
btc.csv
tether.csv
eth.csv
tether.csv
eth.csv
